# Switch calculation engine
This notebook shows how we can switch calculation engine (external crystallographic library) and calculate diffraction profile using constant wavelength experiment type.

In [ ]:
# EasyScience, technique-independent
from easyCore import np
from easyCore.Fitting.Fitting import Fitter

# EasyScience, diffraction
from easyDiffractionLib import Site, Phase, Phases
from easyDiffractionLib.Jobs import Powder1DCW, InterfaceFactory

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

# Misc
import xarray as xr

In [ ]:
output_notebook()
FIGURE_WIDTH = 700
FIGURE_HEIGHT = 300

## Sample

### Create an atom using `Site` interface

In [ ]:
atom = Site(label="Cl",
           specie="Cl",
           fract_x=0.0,
           fract_y=0.0,
           fract_z=0.0)

### Create a phase, set space group, add atom

In [ ]:
phase = Phase(name="salt")

phase.spacegroup.space_group_HM_name = "F m -3 m"

phase.add_atom(atom)

### Add another atom (using `Phase` interface)

In [ ]:
phase.add_atom('Na', 'Na', 0.5, 0.5, 0.5)

### Visualise the structure

In [ ]:
viewer = py3Dmol.view()
viewer.addModel(phase.cif,'cif',{'doAssembly':True,'duplicateAssemblyAtoms':True,'normalizeAssembly':True})
viewer.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol', 'radius': 0.1}})
viewer.addUnitCell()
viewer.replicateUnitCell(2,2,2)
viewer.zoomTo()

### Create Phases object

In [ ]:
phases = Phases()
phases.append(phase)

## Experiment

### Create `Dataset`

In [ ]:
data = xr.Dataset()
x_data = np.linspace(20, 170, 500)

### Create `Job`

In [ ]:
job = Powder1DCW('NaCl', data, phases=phases)

In [ ]:
print(f"Current calculator engine: {job.interface.current_interface_name}")

In [ ]:
print(f"Available calculator engines: {job.interface.available_interfaces}")
print(f"Available calculators for CW: {job.interface.interface_compatability('Npowder1DCWunp')}")

###  Modify instrumental parameters

In [ ]:
job.parameters.resolution_u = 0.1447
job.parameters.resolution_v = -0.4252
job.parameters.resolution_w = 0.3864
job.parameters.resolution_x = 0.0
job.parameters.resolution_y = 0.0

###  Modify pattern parameters

In [ ]:
job.pattern.zero_shift = 0.0
job.pattern.scale = 100.0

## Simulation

### Calculate the profile using the default calculation engine

In [ ]:
_ = job.create_simulation(x_data)
y_data_cryspy = np.array(data['sim_NaCl'])

In [ ]:
print("some Ycalc from CrysPy:\n", y_data_cryspy[315:330])

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_cryspy, legend_label='CW Simulation', color='orangered', line_width=2)
show(fig)

###  Switch calculator

In [ ]:
# The following code generates AttributeError
#calculator.switch('CrysFML')
#_ = job.create_simulation(x_data)

In [ ]:
# Temporary workaround
job.interface = InterfaceFactory(interface_name='CrysFML')

In [ ]:
print(f"Current calculator engine: {job.interface.current_interface_name}")

## Simulation

### Calculate the profile using the calculator engine just selected

In [ ]:
_ = job.create_simulation(x_data)
y_data_crysfml = np.array(data['sim_NaCl'])

In [ ]:
print("some Ycalc from CrysFML:\n", y_data_crysfml[315:330])

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_crysfml, legend_label='CW Simulation', color='olivedrab', line_width=2)
show(fig)